# Spectral Lag Analysis using Discrete Cross-Correlation

This notebook demonstrates how to use `grb_pipeline` to compute spectral lags
between energy bands using the Band (1997) Discrete Cross-Correlation Function (DCCF).

The lag is estimated by fitting an asymmetric Gaussian to the CCF peak, with uncertainties
determined via Monte Carlo simulations following the methodology of
[Ukwatta et al. (2012)](https://doi.org/10.1093/mnras/stu2153).

Authors: Vikas Chand

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import ascii
from grb_pipeline.analysis.temporal import TemporalAnalyzer, SpectralLagResult
from grb_pipeline.analysis.lightcurve import LightCurveData

## 1. Read Multi-band Light Curve Data

We read a Swift BAT light curve with 4 energy bands:
- Band 1: 15-25 keV
- Band 2: 25-50 keV
- Band 3: 50-100 keV
- Band 4: 100-350 keV

In [ ]:
# Read the BAT light curve data
GRB1 = ascii.read('8ms_lc_ascii.dat')

times = np.array(GRB1['col1'])
LC1, LC1_err = np.array(GRB1['col2']), np.array(GRB1['col3'])  # 15-25 keV
LC2, LC2_err = np.array(GRB1['col4']), np.array(GRB1['col5'])  # 25-50 keV
LC3, LC3_err = np.array(GRB1['col6']), np.array(GRB1['col7'])  # 50-100 keV
LC4, LC4_err = np.array(GRB1['col8']), np.array(GRB1['col9'])  # 100-350 keV

dt = times[1] - times[0]
print(f"Time resolution: {dt*1000:.1f} ms")
print(f"Number of bins: {len(times)}")

# Create LightCurveData objects
lc_band1 = LightCurveData(time=times, rate=LC1, rate_err=LC1_err, binsize=dt)
lc_band2 = LightCurveData(time=times, rate=LC2, rate_err=LC2_err, binsize=dt)
lc_band3 = LightCurveData(time=times, rate=LC3, rate_err=LC3_err, binsize=dt)
lc_band4 = LightCurveData(time=times, rate=LC4, rate_err=LC4_err, binsize=dt)

## 2. Compute the Observed DCCF

First we compute the discrete cross-correlation function between two energy bands
within a specified time interval.

In [ ]:
analyzer = TemporalAnalyzer()

# Define analysis time window
tstart = -0.64
tstop = 10.0

# Trim the data
mask = (times >= tstart) & (times <= tstop)
t_src = times[mask]

# Compute DCCF between Band 1 (15-25 keV) and Band 2 (25-50 keV)
offsets, ccf = analyzer.discrete_cross_correlation(
    LC1[mask], LC2[mask], t_src
)

# Plot the observed DCCF
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(offsets, ccf, 'k-', linewidth=0.5)
ax.set_xlabel('Time Lag (s)', fontsize=14)
ax.set_ylabel('Cross-Correlation', fontsize=14)
ax.set_title('Band (1997) DCCF: 15-25 keV vs 25-50 keV')
ax.axvline(0, color='r', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

## 3. Full Spectral Lag with Monte Carlo Errors

The `compute_spectral_lag` method performs the complete pipeline:
1. Computes the observed DCCF
2. Estimates CCF errors via 10,000 Monte Carlo simulations
3. Fits an asymmetric Gaussian to determine the peak lag
4. Estimates lag errors via 1,000 randomised-CCF fits

In [ ]:
# Compute spectral lag with full Monte Carlo error estimation
# Use smaller simulation counts for a quick demo; increase for publication results
result = analyzer.compute_spectral_lag(
    lc_low=lc_band1,
    lc_high=lc_band2,
    tstart=tstart,
    tstop=tstop,
    n_ccf_sims=1000,    # Use 10000 for publication quality
    n_lag_sims=200,      # Use 1000 for publication quality
    data_type='swift',
)

print(f"Spectral Lag (Band 1 -> Band 2): {result.lag*1000:.2f} +/- {result.lag_err*1000:.2f} ms")
print(f"Fit parameters: {result.fit_params}")

In [ ]:
# Plot the CCF with errors and the asymmetric Gaussian fit
fig, ax = plt.subplots(figsize=(10, 5))

ax.errorbar(result.offsets, result.ccf, yerr=result.ccf_err,
            fmt='o', markersize=2, color='steelblue', alpha=0.6,
            elinewidth=0.5, label='DCCF')

# Overlay the asymmetric Gaussian fit
from grb_pipeline.analysis.temporal import TemporalAnalyzer
fit_x = np.linspace(result.offsets.min(), result.offsets.max(), 5000)
fit_y = TemporalAnalyzer.asymmetric_gaussian(
    fit_x, **result.fit_params
)
ax.plot(fit_x, fit_y, 'r-', linewidth=2, label='Asymmetric Gaussian fit')
ax.axvline(result.lag, color='k', linestyle='--', alpha=0.7,
           label=f'Lag = {result.lag*1000:.2f} ms')

ax.set_xlabel('Time Lag (s)', fontsize=14)
ax.set_ylabel('Cross-Correlation', fontsize=14)
ax.set_title('DCCF with Asymmetric Gaussian Fit')
ax.legend(fontsize=11)
plt.tight_layout()
plt.show()

## 4. Summary

The spectral lag between the 15-25 keV and 25-50 keV bands is consistent
with previous results (e.g., ~-0.089 ± 0.014 s at 8 ms resolution).

For publication-quality results, use `n_ccf_sims=10000` and `n_lag_sims=1000`.